In [1]:
import os

In [2]:
%pwd

'd:\\Text_summarization\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text_summarization'

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataValidationConfig:
    root_dir:Path
    status_file:Path
    All_required_file:list

In [37]:
from src.TEXT_SUMMARIZATION.constants import *
from src.TEXT_SUMMARIZATION.common import *

In [44]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            status_file = config.status_file,
            All_required_file = config.All_required_file
        )
        return data_validation_config

In [45]:
import os
import urllib.request as request
import zipfile
from src.TEXT_SUMMARIZATION.logger import logging
from src.TEXT_SUMMARIZATION.common import *

In [48]:
class DataValidation:
    def __init__(self,config:DataValidationConfig):
        self.config = config


    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None
            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.All_required_file:
                    validation_status = False
                    with open(self.config.status_file,'w') as f:
                        f.write(f"Validation status of {file}:{validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file,'w') as f:
                        f.write(f"Validation status of {file} : {validation_status}")
            return validation_status
        
        except Exception as e:
            raise CustomException(e,sys)

In [49]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise CustomException(e,sys)